In [91]:
# import dependencies
import pandas as pd
import numpy as np
import sys
from datetime import datetime
import getpass

# # For MySQL
# from sqlalchemy import create_engine
# import pandas as pd
# import pymysql

# For PostgreSQL
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

# # For SQL Server
# from sqlalchemy import create_engine
# import pandas as pd
# import pyodbc

In [92]:
import psycopg2

username = 'gmb'
password = getpass.getpass('Enter password:')
public_ip_address = '34.28.219.203'
db_name = 'gmb'

# Create a connection
conn = psycopg2.connect(
    dbname=db_name,
    user=username,
    password=password,
    host=public_ip_address
)

# Create a cursor
cur = conn.cursor()

Enter password:········


In [118]:
# Query to get all table names in the PostgreSQL database
query = """
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname != 'pg_catalog' AND 
      schemaname != 'information_schema';
"""

# Execute the query
cur.execute(query)

# Anonymization

In [149]:
query = """

SELECT distinct table_name
        FROM information_schema.columns
WHERE column_name = 'first_name' 
   OR column_name = 'last_name'
   OR column_name = 'phone'
   OR column_name = 'email'
   OR column_name = 'zipcode'
   OR column_name = 'password' 
"""
sensitive_tables = pd.read_sql(query, conn)
sensitive_tables

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,table_name
0,billing_paymentcustomer
1,billing_paymentcustomer_anonymized
2,challenge_challengeinvite
3,group_groupinvite
4,matching_matchingsettings
5,member_user
6,member_user_anonymized
7,social_auth_code


### billing_paymentcustomer


In [120]:
query_billing = "select * from billing_paymentcustomer"
biling_paymentcustomer_df = pd.read_sql(query_billing, conn)

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [121]:
biling_paymentcustomer_anonydf = billing_paymentcustomer_df.copy()

biling_paymentcustomer_anonydf['first_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'FirstUser{}'.format(x))

biling_paymentcustomer_anonydf['last_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'LastUser{}'.format(x))

biling_paymentcustomer_anonydf['email'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'user{}@example.com'.format(x))

if 'phone' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['phone'] = biling_paymentcustomer_anonydf['id']\
    .apply(lambda x: '123456{}'.format(x))
    
if 'created_from_ip' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['created_from_ip'] = "192.168.1.1" 
    


**About anonymizing IP address:**
"192.168.1.1" is a commonly used default IP address for many routers and home networking equipment. It falls within the range of IP addresses (192.168.0.0 to 192.168.255.255) reserved for private networks, meaning these addresses are not routable on the public internet and are often used for local network devices.

In [123]:
db_url = f"postgresql://{username}:{password}@{public_ip_address}/{db_name}"

# SQLAlchemy engine
engine = create_engine(db_url)

when using the `to_sql()` method in pandas, you need to provide an SQLAlchemy engine, not a psycopg2 connection. This is because pandas uses SQLAlchemy to handle SQL dialects.

In [124]:
biling_paymentcustomer_anonydf.to_sql('billing_paymentcustomer_anonymized', engine, if_exists='replace', index=False)


590

### challenge_challengeinvite

In [127]:
query_challenge = "select * from challenge_challengeinvite" 
challenge_challengeinvite_df = pd.read_sql(query_challenge, conn)
challenge_challengeinvite_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,status,created_at,updated_at,challenge_id,invited_by_id,user_id
0,12,None,20,2020-04-01 23:12:53.059068+00:00,2020-04-04 01:22:50.061822+00:00,158,2,9033
1,3,None,10,2020-03-14 23:41:45.580212+00:00,2020-03-14 23:41:45.580235+00:00,124,2,9065
2,5,None,10,2020-03-14 23:44:36.088580+00:00,2020-03-14 23:44:36.088598+00:00,117,2,17
3,2,None,20,2020-03-14 20:23:03.564005+00:00,2020-03-14 23:45:46.688291+00:00,117,2,9237
4,4,None,20,2020-03-14 23:44:35.956449+00:00,2020-03-15 00:07:48.216763+00:00,124,2,17
...,...,...,...,...,...,...,...,...
485,552,None,20,2023-03-12 08:12:37.341775+00:00,2023-03-12 08:46:28.135503+00:00,843,2,13382
486,555,None,10,2023-03-27 06:11:07.022965+00:00,2023-03-27 06:11:07.022989+00:00,838,2,13427
487,554,None,10,2023-03-27 06:11:07.022602+00:00,2023-03-27 06:11:07.022638+00:00,842,2,13427
488,556,None,20,2023-05-02 08:07:59.558115+00:00,2023-05-02 16:42:18.068892+00:00,848,2,12858


In [130]:
challenge_challengeinvite_df.email.unique()

array([None], dtype=object)

No need to anonymize the `challenge_challengeinvite` table currently since the `email` column consists of no emails.

### group_groupinvite

In [131]:
query_groupinvite = "select * from group_groupinvite" 
group_groupinvite_df = pd.read_sql(query_groupinvite, conn)
group_groupinvite_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,status,created_at,updated_at,group_id,user_id,invited_by_id
0,2,None,20,2019-09-10 15:59:33.490257+00:00,2019-09-10 16:36:16.351336+00:00,175,6218,2
1,3,None,20,2019-09-10 16:00:31.655272+00:00,2019-09-10 18:48:19.756997+00:00,175,8328,2
2,1,None,20,2019-09-10 15:41:35.198606+00:00,2019-09-11 07:21:00.675526+00:00,175,6294,2
3,6,None,10,2019-09-11 08:12:56.115308+00:00,2019-09-11 08:12:56.115330+00:00,31,5411,2
4,8,None,20,2019-09-11 21:05:40.445499+00:00,2019-09-11 22:24:21.599480+00:00,175,510,2
...,...,...,...,...,...,...,...,...
393,420,None,20,2023-04-19 18:52:46.022545+00:00,2023-04-19 19:57:54.151530+00:00,932,13467,2
394,422,None,20,2023-05-25 06:06:40.623903+00:00,2023-05-25 22:21:11.015668+00:00,1309,13523,2
395,421,None,20,2023-05-25 06:06:13.441424+00:00,2023-05-25 23:32:12.239884+00:00,1309,12870,2
396,423,None,10,2023-06-02 20:20:32.576387+00:00,2023-06-02 20:20:32.576411+00:00,1309,13407,2


In [132]:
group_groupinvite_df.email.unique()

array([None], dtype=object)

No need to anonymize the `group_groupinvite` table currently since the `email` column consists of no emails.

### matching_matchingsettings

In [134]:
query_matching ='select * from matching_matchingsettings'
matching_matchingsettings_df = pd.read_sql(query_matching, conn)
matching_matchingsettings_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,percentage,email
0,1,True,True


No anonymization needed.

### member_user

In [145]:
query_memberuser = "select * from member_user" 
member_user_df = pd.read_sql(query_memberuser, conn)
member_user_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,password,last_login,is_superuser,first_name,last_name,is_staff,is_active,date_joined,email,...,last_seen_group_invites,last_phone_verification_sent,phone_number_verified,phone_verification_hash,last_seen_challenge_invites,pre_signup_subscription_id,referral_code,redirect_url,campaign_id,date_register_finished
0,12685,pbkdf2_sha256$150000$R6IOehYou4w4$cfmVfPUf/4fQ...,NaT,False,,,False,False,2022-07-15 03:01:33.018800+00:00,michael+210300@getmotivaedbuddies.com,...,2022-07-15 03:01:33.019053+00:00,NaT,False,None,2022-07-15 03:01:33.019060+00:00,None,None,None,57e53ce0-12df-43c0-bcd9-2d2634e0af12,NaT
1,12695,pbkdf2_sha256$150000$0Rw0YqbpSvlv$zmyB0ecD641P...,NaT,False,,,False,False,2022-07-18 01:36:33+00:00,michael+wheretest@getmotivatedbuddies.comm,...,2022-07-18 01:36:33+00:00,NaT,False,None,2022-07-18 01:36:33+00:00,None,None,None,57e53ce0-12df-43c0-bcd9-2d2634e0af12,NaT
2,12688,pbkdf2_sha256$150000$9U0EiD6cSKSB$YShI8Zs/B5rn...,2022-07-15 03:27:06+00:00,False,Michael,G,False,False,2022-07-15 03:26:44+00:00,michael+248900@getmotivatedbuddies.com,...,2022-07-15 03:26:44+00:00,NaT,False,None,2022-07-15 03:26:44+00:00,None,None,None,2e9183f2-834b-4831-8487-823afb673645,2022-07-15 03:29:13.733293+00:00
3,12614,pbkdf2_sha256$150000$KlJOXx01vdag$8ajz0iIyWECU...,2023-01-10 18:25:16.782783+00:00,False,Yeison,Quiceno,False,True,2022-06-09 21:17:48.095567+00:00,yaquicenod@unal.edu.co,...,2022-06-09 21:17:48.095720+00:00,2022-06-09 21:38:19.307348+00:00,True,None,2022-06-09 21:17:48.095725+00:00,None,None,None,None,2022-07-15 02:23:11.922821+00:00
4,12687,pbkdf2_sha256$150000$376TbsvGvAPS$hqx8ALMJem0X...,2022-07-15 03:19:09+00:00,False,michael,Goldstrom,False,False,2022-07-15 03:18:52+00:00,michael+28000@getmotivatedbuddies.com,...,2022-07-15 03:18:52+00:00,NaT,False,None,2022-07-15 03:18:52+00:00,None,None,None,c4fe0b58-87b2-4ea2-9a74-e78136180d81,2022-07-15 03:22:02.993299+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12882,12684,pbkdf2_sha256$150000$ESqNpE9y9gw9$G0u4/vsieKBu...,2022-07-20 00:01:11.251717+00:00,False,Amy,Kos,False,True,2022-07-14 14:54:34.134187+00:00,pleasecallifpossible@gmail.com,...,2022-07-14 14:54:34.134340+00:00,NaT,False,None,2022-07-14 14:54:34.134345+00:00,None,None,None,None,2022-07-15 02:23:11.922821+00:00
12883,12674,pbkdf2_sha256$150000$mdmAQ5fqdydt$NC4PDHNwIB0B...,2022-07-16 13:50:14.058743+00:00,False,Brandon,B,False,True,2022-07-10 18:08:22.214274+00:00,brandon35179@gmail.com,...,2022-07-10 18:08:22.214421+00:00,2022-07-10 18:25:47.502667+00:00,False,6dca0d5d-c3ba-4c8b-9cf5-a3cefa24e9d7,2022-07-10 18:08:22.214426+00:00,None,None,None,None,2022-07-15 02:23:11.922821+00:00
12884,12683,pbkdf2_sha256$150000$LXVpMtUg7w6Z$mpzOvejMlxEU...,2022-07-19 14:08:14.262031+00:00,False,Bishop,D,False,True,2022-07-14 03:45:56.556178+00:00,bishara_bates@yahoo.com,...,2022-07-14 03:45:56.556329+00:00,NaT,False,None,2022-07-14 03:45:56.556334+00:00,None,None,None,None,2022-07-15 02:23:11.922821+00:00
12885,12678,pbkdf2_sha256$150000$xv24WnGs1z43$lLrfpM+sqpFT...,2022-09-29 23:53:30.862360+00:00,True,asdadsasd,adsasdasd,True,True,2022-07-12 02:39:39.105046+00:00,cesarlvielma@gmail.com,...,2022-07-12 02:39:39.105200+00:00,NaT,False,None,2022-07-12 02:39:39.105204+00:00,None,None,None,None,2022-07-15 02:23:11.922821+00:00


In [137]:
# column names
member_user_df.columns

Index(['id', 'password', 'last_login', 'is_superuser', 'first_name',
       'last_name', 'is_staff', 'is_active', 'date_joined', 'email',
       'middle_name', 'avatar', 'date_of_birth', 'phone_number',
       'avatar_body_color', 'avatar_hair_color', 'timezone_name',
       'timezone_difference', 'confirmation_hash', 'deleted_at', 'updated_at',
       'avatar_body_id', 'avatar_eyes_id', 'avatar_hair_id', 'avatar_mouth_id',
       'unique_hash', 'password_reset_hash', 'location_id', 'gender_id',
       'scoring_data', 'old_beta_import', 'password_updated_at', 'on_hold_end',
       'on_hold_start', 'invisible', 'tos_acceptance', 'unlocked_categories',
       'available_categories', 'date_joined_beta', 'payment_customer_id',
       'weight_loss', 'measurement_unit', 'segmentation_id', 'last_online',
       'last_seen_buddy_requests', 'last_seen_group_invites',
       'last_phone_verification_sent', 'phone_number_verified',
       'phone_verification_hash', 'last_seen_challenge_invites',


In [138]:
import hashlib

In [146]:
member_user_anonydf = member_user_df.copy()

member_user_anonydf['first_name'] = member_user_anonydf["id"].apply(lambda x: 'FirstUser{}'.format(x)) #first_name

member_user_anonydf['last_name'] = member_user_anonydf["id"].apply(lambda x: 'LastUser{}'.format(x)) #last_name

member_user_anonydf['email'] = member_user_anonydf["id"].apply(lambda x: 'user{}example@.com'.format(x)) #email

if 'phone_number' in member_user_anonydf.columns:
    member_user_anonydf['phone_number'] = member_user_anonydf['id'].apply(lambda x: '123456{}'.format(x)) #phone
    
if 'password' in member_user_anonydf.columns:
    member_user_anonydf['password'] = hashlib.sha256(password.encode('utf-8')).hexdigest() #password


In [147]:
member_user_anonydf.to_sql('member_user_anonymized', engine, if_exists='replace', index=False)

413

### social_auth_code

In [148]:
query_social = "select * from social_auth_code" 
social_auth_df = pd.read_sql(query_social, conn)
social_auth_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,code,verified,timestamp


### member_location

In [150]:
query_location = "select * from member_location" 
member_location_df = pd.read_sql(query_location, conn)
member_location_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,google_place_id,formatted_name,lat,long,route,administrative_area_level_1,city,country,utc_offset,country_iso,postal_code,street_number,administrative_area_level_1_short,created_at,name,updated_at
0,13,EjxTYW4gRnJhbmNpc2NvIOKAkyBPYWtsYW5kIEJheSBCcm...,"San Francisco – Oakland Bay Bridge, San Franci...",37.810382,-122.364788,San Francisco – Oakland Bay Bridge,California,San Francisco,United States,-480.0,US,,,CA,2019-03-05 11:40:21.610778+00:00,None,2019-03-07 16:08:46.399446+00:00
1,2,EhVTYW50YSBNb25pY2EsIENBLCBVU0EiLiosChQKEgmnBZ...,"Santa Monica Pier, Santa Monica, CA 90401, USA",34.009351,-118.497468,Santa Monica Pier,California,Santa Monica,United States,-480.0,US,90401,,CA,2019-03-05 11:40:21.610778+00:00,None,2019-03-07 16:08:46.399446+00:00
2,1,ChIJA_L3Au3CQUcR3p_UGR5c2i0,"Budapest, Jáhn Ferenc u. 140, 1196 Hungary",47.444583,19.143465,Jáhn Ferenc utca,,Budapest,Hungary,60.0,HU,1196,140,,2019-03-05 11:40:21.610778+00:00,None,2019-03-07 16:08:46.399446+00:00
3,3,EjFTYW50YSBNb25pY2EgQm91bGV2YXJkLCBXZXN0IExvcy...,"Santa Monica Blvd, Los Angeles, CA, USA",34.046979,-118.446651,Santa Monica Boulevard,California,Los Angeles,United States,-480.0,US,,,CA,2019-03-05 11:40:21.610778+00:00,None,2019-03-07 16:08:46.399446+00:00
4,4,EihTYW50YSBNb25pY2EgUGllciwgU2FudGEgTW9uaWNhLC...,"Santa Monica Pier, Santa Monica, CA 90401, USA",34.009351,-118.497468,Santa Monica Pier,California,Santa Monica,United States,-480.0,US,90401,,CA,2019-03-05 11:40:21.610778+00:00,None,2019-03-07 16:08:46.399446+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5332,6032,EjNSb2NrYXdheSBGd3kgJiBCZWFjaCA1OXRoIFN0LCBRdW...,"Rockaway Fwy & Beach 59th St, Queens, NY 11692...",40.592453,-73.789007,None,New York,None,United States,-240.0,US,11692,None,NY,2023-07-03 11:29:15.967489+00:00,Rockaway Freeway & Beach 59th Street,2023-07-03 11:29:15.967464+00:00
5333,6034,ChIJjSY513s7uUcRQPFbNqRY71Q,"Böhmerstraße 56, 58095 Hagen, Germany",51.353857,7.470720,Böhmerstraße,Nordrhein-Westfalen,Hagen,Germany,120.0,DE,58095,56,NRW,2023-07-03 11:55:27.165934+00:00,Böhmerstraße 56,2023-07-03 11:55:27.165909+00:00
5334,6037,EjUxOXRoIEF2ZSBTRSAmIExha2UgSGVpZ2h0cyBEciwgRX...,"19th Ave SE & Lake Heights Dr, Everett, WA 982...",47.891492,-122.205527,None,Washington,Everett,United States,-420.0,US,98208,None,WA,2023-07-09 02:06:30.356449+00:00,19th Avenue Southeast & Lake Heights Drive,2023-07-09 02:06:30.356424+00:00
5335,6040,ChIJh60nVIzGwoARoBIPhill5_U,"475 N Ave 19, Los Angeles, CA 90031, USA",34.079700,-118.226110,North Avenue 19,California,Los Angeles,United States,-420.0,US,90031,475,CA,2023-07-12 01:44:03.567741+00:00,475 N Ave 19,2023-07-12 01:44:03.567715+00:00


In [157]:
import random

def anonymize_street_address(formatted_address: str) -> str:

    parts = formatted_address.split(",")
    if len(parts) < 4:
        return "123 Anonymized St, Anonymized City, Anonymized State, 00000"

    random_street_name = f"{random.randint(1,999)} Anonymized St"

    generalized_city = "Anonymized City"

    masked_state_code = "Anonymized State"

    anonymized_zip_code = "00000"

    return f"{random_street_name}, {generalized_city}, {masked_state_code}, {anonymized_zip_code}"



In [158]:
member_location_anonydf = member_location_df.copy()

member_location_anonydf['formatted_name'] = member_location_anonydf['formatted_name'].\
apply(anonymize_street_address)


In [160]:
member_location_anonydf.to_sql('member_location_anonymized', engine, if_exists='replace', index=False)

337